In [6]:

##
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
import calendar
import numpy as np

# loading dataframes 

url_tsfactor = 'https://github.com/Sebasleen/Seminargroup/raw/Seminar/Data/TSFactor.dta'
tsfactor = pd.read_stata(url_tsfactor)[['year', 'month', 'TSMom', 'yyyymm']]
tsfactor['date'] = pd.to_datetime(tsfactor['yyyymm'], format='%Y%m')
# Set the 'date' column as the index
tsfactor.set_index('date', inplace=True)

# Now you can format the index with strftime
tsfactor.index = tsfactor.index.strftime('%Y-%m')

y_monthly_n = tsfactor['TSMom']

r_daily = pd.read_csv('managed_portfolios_anom_d_55.csv')
r_daily['date'] = pd.to_datetime(r_daily['date'])
r_daily.set_index('date', inplace=True)

y_daily = r_daily['r_mom']


# drop all momentum factors or factors that are constructed based on momentum

factor_drop_list = ['r_mom', 'r_indmom', 'r_valmom', 'r_valmomprof', 'r_mom12', 'r_momrev', 'r_indmomrev', 'r_exchsw', 'rme', 're_ew']

r_daily.drop(columns=factor_drop_list, inplace=True)


# following the procedure in the paper, if there are observations missing we set them to 0. 

r_daily.fillna(0, inplace=True)

# create a list of factors for later analysis purposes 

factors = [col for col in r_daily.columns if col.startswith('r_')]

# create a monthly return dataframe for later analysis purposes 

r_monthly = r_daily.resample('ME').sum()
r_monthly.index = r_monthly.index.strftime('%Y-%m')
y_monthly = y_daily.resample('ME').sum()
y_monthly.index = y_monthly.index.strftime('%Y-%m')



In [ ]:


# select our start date 

start_date = pd.to_datetime("1963-07-01")

pls_avg_df = pd.DataFrame()

# create an empty list for the pls return dataframes. These will be concated in a later stage to one large dataframe

pls_return_dfs = []


# Create an empty dataframe for monthly PLS returns
pls_return_df = pd.DataFrame(columns=['date'] + [f"PLS{i+1}" for i in range(len(factors))])

# Initialize an empty list to store PLS weights
pls_weights_list = []

# Loop through years and months
for year in range(1973, 2020):
    # as our out of sample procedure starts July 1973, we start in the 7th month in 1973
    for mo in range(6,13) if year == 1973 else range(1, 13):
        # first we have to find the last month of the day. For this we use the calender function with inputs from the loop variables
        last_day = calendar.monthrange(year, mo)[1]

        # we select our new end_date variable for which the Pls analysis is done, also with inputs from our loop and the last_day variable
        end_date = pd.to_datetime(f'{year}-{mo}-{last_day}')

        t_dt = pd.to_datetime(f'{year}-{mo}')
        t = t_dt.strftime('%Y-%m')

        # Select data range for PLS analysis
        pls_data = r_daily.loc[start_date:end_date]
        pls_data_y = y_daily.loc[start_date:end_date]

        # Separate features (X) and target (y)
        X = pls_data
        y = pls_data_y

        # Fit PLS model
        pls = PLSRegression(n_components=47)
        pls.fit(X, y)


        # Extract and store PLS weights
        pls_weights = pd.DataFrame(
            np.array(pls.x_weights_),
            index=factors,
            columns=[f"PLS{i+1}" for i in range(47)]
        )
        # calculating return for month t+1. If mo = 12, then year will increment with 1. 

        t_plus_1_year = year + 1 if mo == 12 else year
        t_plus_1_month = (mo % 12) + 1

        # creating a datetime variable for the month t+1 and storing this in our pls_return_data variable

        t_plus_1_dt =pd.to_datetime(f'{t_plus_1_year}-{t_plus_1_month}')
        t_plus_1 = t_plus_1_dt.strftime('%Y-%m')

        pls_return_data = {'date': t_plus_1}


        # in this loop we calculate the monthly factor returns (f) using the principal components and returns

        for f in range(len(factors)):
            # select our factor and extract its principal component from principal_df and its return from r_daily for all observations in month mo 
            pls = pls_weights.iloc[:, f]
            r_month = r_monthly.loc[t]

            # multiply the principal components with the returns and sum them up to get Pls factor return for month mo 
            pls_return = (pls*r_month).sum()

            # place this in our dictionary for later transposing to dataframe

            pls_return_data[pls_weights.columns[f]] = pls_return

            r_pls_month_n_list = []

            for n in range(1, 12):
                # calculate the datetime for t - n
                t_minus_n_dt = t_dt - pd.DateOffset(months=n)

                # transpose it to our YYYY-MM format
                t_minus_n = t_minus_n_dt.strftime('%Y-%m')

                # select the return corresponding to our month t-n
                r_month_n = r_monthly.shift(n).loc[t_minus_n]
                pls_return_n = (pls*r_month_n).sum()
                r_pls_month_n_list.append(pls_return_n)

            r_pls_month_mean = (np.mean(r_pls_month_n_list))
            pls_avg_df.loc[t, f'PLS{f+1}'] = r_pls_month_mean


        pls_return_df = pd.DataFrame.from_dict(pls_return_data, orient='index').T
        pls_return_df.set_index('date', inplace=True)
        pls_return_dfs.append(pls_return_df)

r_pls = pd.concat(pls_return_dfs)
print(r_pls)
print(pls_avg_df)


In [194]:

# Define the start date
start_date_dt = pd.to_datetime("1963-07-01")
start_date = start_date_dt.strftime('%Y-%m')

# Create an empty list to store leverage-adjusted PLS factors
lev_pls_dfs = []

for year in range(1973, 2020):
    # Loop through months
    for mo in range(7, 13) if year == 1973 else range(1, 13):
        # Define the current date
        t_dt = pd.to_datetime(f'{year}-{mo}')
        t = t_dt.strftime('%Y-%m')

        # Calculate the variance of the individual factor returns up until month t 
        r_indiv_f_t = r_monthly.loc[start_date:t]
        var_indiv_f_t = r_indiv_f_t.var(axis=0)
        avg_var_indiv_f_t = var_indiv_f_t.mean()

        # Calculate the mean and variance of the PLS factors up until month t 
        r_pls_t = r_pls.loc[:t]
        demeaned_r_pls_t = r_pls_t.loc[t].to_frame().T - r_pls_t.mean()

        # Calculate the leverage factor
        leverage_t = np.sqrt(avg_var_indiv_f_t / r_pls_t.var(axis=0))

        # Multiply leverage factor with the demeaned PLS factors
        lev_r_pls_t = demeaned_r_pls_t * leverage_t

        # Append the leveraged PLS factors to the list
        lev_pls_dfs.append(lev_r_pls_t)

# Concatenate the leveraged PLS factors into a single DataFrame
lev_r_pls = pd.concat(lev_pls_dfs)
lev_r_pls.fillna(0, inplace=True)
lev_r_pls_clean = lev_r_pls.drop(lev_r_pls.index[:1])

print(lev_r_pls_clean)

Some conditions might not be met. Investigate further.
PLS1     0.000823
PLS2    -0.001276
PLS3     0.000675
PLS4     0.010758
PLS5    -0.002898
PLS6     0.000298
PLS7    -0.001293
PLS8    -0.004450
PLS9    -0.000069
PLS10    0.004192
PLS11    0.001809
PLS12   -0.001431
PLS13   -0.001688
PLS14    0.000557
PLS15   -0.003934
PLS16   -0.001967
PLS17   -0.003361
PLS18    0.000703
PLS19   -0.005010
PLS20   -0.001797
PLS21   -0.003605
PLS22    0.001662
PLS23   -0.005580
PLS24   -0.004361
PLS25   -0.000108
PLS26    0.001353
PLS27   -0.003533
PLS28    0.001919
PLS29   -0.000385
PLS30    0.002888
PLS31   -0.002937
PLS32    0.006248
PLS33    0.003680
PLS34   -0.000517
PLS35    0.000251
PLS36    0.002222
PLS37   -0.000095
PLS38    0.007622
PLS39   -0.001885
PLS40   -0.004254
PLS41    0.004402
PLS42   -0.000103
PLS43   -0.001350
dtype: float64


In [184]:
#mergen
# Define the start date
start_date_dt = pd.to_datetime("1963-07-01")
start_date = start_date_dt.strftime('%Y-%m')

# Create an empty list to store leveraged individual factor momentum returns
indiv_lev_dfs = []

for year in range(1973, 2018):
    # Loop through months
    for mo in range(7, 13) if year == 1973 else range(1, 13):
        # Define the current date
        t_dt = pd.to_datetime(f'{year}-{mo}')
        t = t_dt.strftime('%Y-%m')

        # Calculate the variance of the individual factor returns up until month t 
        r_indiv_f_t = r_monthly.loc[start_date:t]
        var_indiv_f_t = r_indiv_f_t.var(axis=0)
        avg_var_indiv_f_t = var_indiv_f_t.mean()

        # Calculate the leverage factor 
        leverage_factor = np.sqrt(avg_var_indiv_f_t / var_indiv_f_t)

        # Multiply the leverage factor with the returns
        indiv_lev_df = r_monthly.loc[t].to_frame().T * leverage_factor

        # Append the leveraged individual factor momentum returns to the list
        indiv_lev_dfs.append(indiv_lev_df)

# Concatenate the leveraged individual factor momentum returns into a single DataFrame
lev_r_indiv = pd.concat(indiv_lev_dfs)
lev_r_indiv.fillna(0, inplace=True)

# Print the DataFrame
print(lev_r_indiv)

           r_size   r_value    r_prof  r_valprof  r_fscore  r_debtiss  \
1973-07 -0.048953 -0.125946  0.129852   0.033222  0.031672  -0.303715   
1973-08  0.020838  0.009044 -0.005810  -0.016223  0.035741  -0.004212   
1973-09 -0.037885  0.016908  0.009182  -0.005630  0.085452  -0.205952   
1973-10 -0.005048  0.044587 -0.034781   0.003408 -0.006801  -0.085912   
1973-11  0.087170  0.081589 -0.126876  -0.068979 -0.102567   0.162956   
...           ...       ...       ...        ...       ...        ...   
2017-08  0.025351 -0.029649  0.036218  -0.024006 -0.004275   0.047617   
2017-09 -0.036670  0.021026 -0.040234  -0.010097 -0.050019   0.001135   
2017-10 -0.005193 -0.011033  0.018336  -0.007419  0.045026   0.029219   
2017-11  0.008796 -0.013538  0.042746   0.050914  0.016230   0.015697   
2017-12  0.009780 -0.001831  0.006061   0.003697 -0.055648   0.014643   

         r_repurch   r_nissa  r_accruals  r_growth  ...   r_strev    r_ivol  \
1973-07   0.173426  0.015890    0.028854  0.

In [192]:
positive_returns_PLS = pls_avg_df > 0
negative_returns_PLS = pls_avg_df < 0


long_portfolio_PLS = positive_returns_PLS.astype(int)
short_portfolio_PLS = negative_returns_PLS.astype(int)

mom_1_10 = ['PLS1', 'PLS2', 'PLS3', 'PLS4', 'PLS5', 'PLS6', 'PLS7', 'PLS8', 'PLS9', 'PLS10']
mom_11_20 = ['PLS11', 'PLS12', 'PLS13', 'PLS14', 'PLS15', 'PLS16', 'PLS17', 'PLS18', 'PLS19', 'PLS20']
mom_21_30 = ['PLS21', 'PLS22', 'PLS23', 'PLS24', 'PLS25', 'PLS26', 'PLS27', 'PLS28', 'PLS29', 'PLS30']
mom_31_40 = ['PLS31', 'PLS32', 'PLS33', 'PLS34', 'PLS35', 'PLS36', 'PLS37', 'PLS38', 'PLS39', 'PLS40']
mom_41_47 = ['PLS41', 'PLS42', 'PLS43', 'PLS44', 'PLS45', 'PLS46', 'PLS47']

mom_list = [mom_1_10, mom_11_20, mom_21_30, mom_31_40, mom_41_47]

r_mean_set_dict = {}

for i, mom in enumerate(mom_list):
    r_PLS_set_mom = (long_portfolio_PLS[mom] * lev_r_pls_clean[mom]) - (short_portfolio_PLS[mom] * lev_r_pls_clean[mom])
    r_PLS_set_mean = r_PLS_set_mom.mean(axis=1)
    r_mean_set_dict[f'mom_set_{i + 1}'] = r_PLS_set_mean

mom_strategy = pd.concat(r_mean_set_dict, axis=1)

mom_strategy.index = pd.to_datetime(mom_strategy.index)
mom_strategy.index = mom_strategy.index.strftime('%Y-%m')
mom_strategy.dropna(inplace=True)
print(mom_strategy)
print(mom_strategy.mean(axis=0))



1973-06         NaN
1973-07    0.000000
1973-08    0.000000
1973-09    0.110818
1973-10    0.122549
             ...   
2017-08   -0.072997
2017-09    0.098384
2017-10    0.103905
2017-11    0.289313
2017-12    0.097110
Length: 535, dtype: float64


In [189]:
mom_strategy_1 = mom_strategy.loc[:'1996-09']
mom_strategy_2 = mom_strategy.loc['1996-09':]

print(f'the mean of every subset of PCs is:\n')
print(mom_strategy.mean(axis=0))

means = mom_strategy.mean(axis=0).tolist()
std = mom_strategy.std(axis=0).tolist()
N = mom_strategy.shape[0]

print(f'the t-statistic of every subset of PCS is:\n')
for m, s in zip(means, std):
    t_statistic = m / (s / (N**0.5))
    print(t_statistic)

print(f'the mean of every subset of PCs is (first half):\n')
print(mom_strategy_1.mean(axis=0))

means = mom_strategy_1.mean(axis=0).tolist()
std = mom_strategy_1.std(axis=0).tolist()
N = mom_strategy_1.shape[0]

print(f'the t-statistic of every subset of PCS is (first half):\n')
for m, s in zip(means, std):
    t_statistic = m / (s / (N**0.5))
    print(t_statistic)

print(f'the mean of every subset of PCs is (second half):\n')
print(mom_strategy_2.mean(axis=0))

means = mom_strategy_2.mean(axis=0).tolist()
std = mom_strategy_2.std(axis=0).tolist()
N = mom_strategy_2.shape[0]

print(f'the t-statistic of every subset of PCS is (second half):\n')
for m, s in zip(means, std):
    t_statistic = m / (s / (N**0.5))
    print(t_statistic)

                0   Average
1973-06       NaN       NaN
1973-07  0.000000  0.000000
1973-08  0.000000  0.000000
1973-09  0.110818  0.110818
1973-10  0.122549  0.122549
...           ...       ...
2017-08 -0.072997 -0.072997
2017-09  0.098384  0.098384
2017-10  0.103905  0.103905
2017-11  0.289313  0.289313
2017-12  0.097110  0.097110

[535 rows x 2 columns]


In [2]:
ff = pd.read_stata("fffactors.dta")

ff.set_index('yyyymm', inplace=True)


ff.index = pd.to_datetime(ff.index, format='%Y%m')


ff.index = ff.index.strftime('%Y-%m')

ff5 = ff[['mktrf', 'smb', 'hml', 'rmw', 'cma']].loc['1973-08':'2019-12']

mom_strategy_ff5 = pd.concat([mom_strategy, ff5], axis=1)

mom_strategy_ff5['P1'] = 0
mom_strategy_ff5['P2'] = 0
mom_strategy_ff5.loc[mom_strategy_ff5.index <= '1996-09', 'P1'] = 1
mom_strategy_ff5.loc[mom_strategy_ff5.index >= '1996-09', 'P2'] = 1


NameError: name 'r_monthly' is not defined

In [ ]:
independent_vars = ['mom_set_1', 'P1', 'P2', 'mktrf', 'smb', 'hml', 'rmw', 'cma']
X = mom_strategy_ff5[independent_vars]

for i in range(2, 6):
    Y = mom_strategy_ff5[f'mom_set_{i}']
    modelB = sm.OLS(Y, X).fit()
    print(modelB.summary())

In [ ]:
y = mom_strategy_ff5['mom_set_1']

for i in range(2, 6):
    independent_vars = [f'mom_set_{i}', 'P1', 'P2', 'mktrf', 'smb', 'hml', 'rmw', 'cma']
    X = sm.add_constant(mom_strategy_ff5[independent_vars])
    modelC = sm.OLS(y, X).fit()
    print(modelC.summary())


independent_vars = ['P1', 'P2', 'mom_set_2', 'mom_set_3', 'mom_set_4', 'mom_set_5', 'mktrf', 'smb', 'hml', 'rmw', 'cma']

X = sm.add_constant(mom_strategy_ff5[independent_vars])
modelC1 = sm.OLS(y, X).fit()
print(modelC1.summary())
   

In [ ]:
X = sm.add_constant(mom_strategy_ff5[independent_vars])
modelC1 = sm.OLS(y, X).fit()
print(modelC1.summary())